# CAISO-IRP23 Network Model - Execution

This notebook executes the CAISO IRP 2023 energy system model using PyPSA.

## Model Overview

**CAISO IRP23** (California ISO Integrated Resource Plan 2023) represents California's electricity system including:
- Generation resources (thermal, renewables, storage)
- Transmission infrastructure
- Load demand patterns
- Operational constraints

This model uses the **aggregate node strategy** where all demand is assigned to a single `CAISO_Load_Aggregate` node.

## Configuration

- **Solve Duration**: 1 year (to minimize computation time)
- **Output File**: `solved_network_1year.nc`
- **Model Type**: Electricity-only network
- **Demand Strategy**: Aggregate node

---

## Setup and Imports

In [ ]:
import sys
import time
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / "src"))


from plexos_to_pypsa_converter.workflow import run_model_workflow

## Model Configuration

We'll configure the CAISO-IRP23 model to solve for **1 year only** to reduce computation time. The model will:

1. Load the PLEXOS XML data and time series
2. Convert to PyPSA format with aggregate node strategy
3. Solve the optimization problem
4. Save results to a NetCDF file

**Output Location**: `src/examples/results/caiso-irp23/solved_network_1year.nc`

In [ ]:
# Configuration
model_id = "caiso-irp23"
output_filename = "solved_network_1year.nc"
results_dir = Path.cwd().parent / "examples" / "results" / model_id
output_path = results_dir / output_filename

print(f"Model ID: {model_id}")
print(f"Output file: {output_path}")
print(f"Results directory: {results_dir}")

## Execute Model Solve

This cell will execute the complete workflow:
- Load PLEXOS data
- Create PyPSA network
- Run optimization
- ~~Save results~~ (skipped - we'll save manually with custom filename below)

**Note on Network Saving:**

By default, `run_model_workflow()` automatically saves the network to `src/examples/results/[model]/solved_network.nc`. 

To avoid duplicate files, we modify the workflow to **skip the automatic save step** and instead save once with our custom filename (`solved_network_1year.nc`). This gives us better control over the output filename while avoiding unnecessary duplicate saves.

**Note**: This may take several minutes depending on the model size and your system. The solve log will be displayed below.

In [ ]:
# Track solve time
start_time = time.time()

print("=" * 70)
print(" Starting CAISO-IRP23 Model Solve")
print("=" * 70)
print()

# Create modified workflow that skips the automatic save step
# We'll save manually with a custom filename below
from plexos_to_pypsa_converter.db.registry import MODEL_REGISTRY

default_workflow = MODEL_REGISTRY[model_id]["processing_workflow"]
modified_workflow = default_workflow.copy()

# Remove the save_network step (we'll save manually with custom filename)
modified_workflow["steps"] = [
    step for step in default_workflow["steps"] if step["name"] != "save_network"
]

print("Modified workflow: removed automatic save step (will save manually)")
print()

# Run the workflow without automatic save
# Note: The run_model_workflow function loads the model, solves it
# but now skips the automatic save to src/examples/results/[model]/solved_network.nc
network, setup_summary = run_model_workflow(
    model_id, workflow_overrides=modified_workflow
)

# Calculate solve time
solve_time = time.time() - start_time

print()
print("=" * 70)
print(" Model Solve Complete")
print("=" * 70)
print(f"Total time: {solve_time / 60:.2f} minutes")

## Save Results

Now we'll save the solved network to our custom filename. This is the **only save** - we skipped the automatic save step above to avoid creating duplicate files.

In [ ]:
# Save network to specified output file
results_dir.mkdir(parents=True, exist_ok=True)
network.export_to_netcdf(str(output_path))

print(f"✓ Network saved to: {output_path}")
print(f"  File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")

## Solve Summary and Verification

Let's verify the solve succeeded and display key statistics.

In [ ]:
# Display network statistics
print("=" * 70)
print(" Network Statistics")
print("=" * 70)
print()
print(f"Buses:           {len(network.buses)}")
print(f"Generators:      {len(network.generators)}")
print(f"Loads:           {len(network.loads)}")
print(f"Links:           {len(network.links)}")
print(f"Storage Units:   {len(network.storage_units)}")
print(f"Stores:          {len(network.stores)}")
print()
print(f"Snapshots:       {len(network.snapshots)}")
print(f"Carriers:        {len(network.carriers)}")
print()

# Display objective value if available
try:
    objective = network.objective
    print(f"Objective Value: ${objective:,.0f}")
except:
    print("Objective Value: Not available")

print()
print(f"Solve Time:      {solve_time / 60:.2f} minutes")
print()
print("=" * 70)

## Carrier Overview

Display the unique carriers (energy types) in the model:

In [ ]:
# Show unique carriers
carriers = network.generators.carrier.unique().tolist()
print(f"Total unique carriers: {len(carriers)}")
print()
print("Carriers in model:")
for i, carrier in enumerate(sorted(carriers), 1):
    print(f"  {i:2d}. {carrier}")

---

## Next Steps

✓ Model solved successfully!

✓ Results saved to: `solved_network_1year.nc`

**Continue to**: `caiso_analysis.ipynb` for detailed analysis and visualization of these results.

---